# <b> Buy and Hold </b>

Example showing the backtesting of a Buy and Hold strategy.

When the strategy starts, it will buy 50% of AAPL and 50% of GOOG.
As soon as the backtest completes, it will return a dataframe 'result' containing the results, which can be used to visualize the equity's evolution and other metrics, like when buy or sell orders are placed.

## <b> Import Libraries </b>

In [1]:
# Import this library
from nbacktest import Backtest, Strategy

# Import other libraries
import yfinance as yf
import warnings

# Ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## <b> Create Strategy </b>

In [2]:
class TestStrategy (Strategy):

    def on_start(self):

        print("Strategy started!")
        
        quantity1 = int((self.balance/2)/self.prices["AAPL"])
        quantity2 = int((self.balance/2)/self.prices["GOOG"])

        order1 = self.buy("AAPL", quantity1)
        order2 = self.buy("GOOG", quantity2)

        trade1 = self.create_trade(orders=[order1, order2], notes="Buy and Hold Trade")

        print("Bought %s AAPL and %s GOOG" % (quantity1, quantity2))


    def on_end(self):
        """"
        This function is called when the backtest of all candles is finished
        """
        print("Strategy finished!")


    def next(self):
        """"
        This function is called every candle (iteration)
        """
        #print("Backtest iteration:", self.iteration)
        pass


## <b> Download OHLC data from Yahoo Finance </b>

In [3]:
universe = ["AAPL", "GOOG"] # List containing the tickers of all the assets you are backtesting

df_ohlc = yf.download(universe, start="2018-01-01", end="2023-01-01", interval="1d") # Download df_ohlc data from Yahoo Finance

print(df_ohlc.shape)
df_ohlc.tail()

[*********************100%***********************]  2 of 2 completed

(1259, 10)


Price            Close                   High                    Low  \
Ticker            AAPL       GOOG        AAPL       GOOG        AAPL   
Date                                                                   
2022-12-23  129.900269  89.199585  130.451943  89.487615  127.713261   
2022-12-27  128.097488  87.332359  129.456984  88.891688  126.806960   
2022-12-28  124.166779  85.872353  129.082614  87.918349  123.999307   
2022-12-29  127.683739  88.345428  128.540804  88.757609  125.831682   
2022-12-30  127.998978  88.126923  128.018685  88.226241  125.536140   

Price                        Open               Volume            
Ticker           GOOG        AAPL       GOOG      AAPL      GOOG  
Date                                                              
2022-12-23  87.024475  128.974237  87.024475  63814900  17815000  
2022-12-27  86.940047  129.427431  88.702977  69007800  15470900  
2022-12-28  85.782968  127.742826  86.905285  85438400  17879600  
2022-12-29  86.398751  126.087812  86.438480  75703700  18280700  
2022-12-30  86.438473  126.501579  86.771195  77034200  19190300

## <b> Run Backtest </b>

In [4]:
bt = Backtest(data=df_ohlc,
              universe=universe,
              strategy_class=TestStrategy,
              price_column="Close",
              cash=100_000
             )

result = bt.run()

result.head()

Strategy started!
Bought 1239 AAPL and 945 GOOG
Strategy finished!


,BALANCE,POSITIONS_TOTAL,EQUITY
ITERATION,,,
2018-01-02,37.180737,99962.819263,100000.000000
2018-01-03,37.180737,100774.440376,100811.621113
2018-01-04,37.180737,101190.526028,101227.706764
2018-01-05,37.180737,102505.027462,102542.208199
2018-01-08,37.180737,102537.476349,102574.657085


## <b> Backtest Statistics </b>

In [5]:
print("---------------------------")
print("Final balance: %.2f" % bt.strategy.balance)
print("Final equity: %.2f" % bt.strategy.equity)
print("---------------------------")

---------------------------
Final balance: 37.18
Final equity: 241907.86
---------------------------


In [6]:
bt.get_trade_statistics()

{'n_won': 1,
 'n_lost': 0,
 'n_total': 1,
 'win_rate': 1.0,
 'mean_profit': 141907.85592269897,
 'median_profit': 141907.85592269897,
 'mean_profit_win': 141907.85592269897,
 'mean_profit_loss': nan,
 'median_profit_win': 141907.85592269897,
 'median_profit_loss': nan,
 'std_profit': nan,
 'mad_profit': 0.0,
 'skewness': nan,
 'kurtosis': nan,
 'mean_median_gap': 0.0,
 'mean_profit_over_vol': nan,
 'median_profit_over_vol': nan}

In [7]:
bt.get_equity_statistics_dollar()

{'total_profit': 141907.85592269897,
 'mean_profit': 112.80433698147772,
 'median_profit': 256.158239364624,
 'std_profit': 4002.17111971636,
 'mad_profit': 2424.4591722933087,
 'skewness': -0.19347760951126758,
 'kurtosis': 3.057028962106326,
 'mean_median_gap': -143.3539023831463,
 'mean_profit_over_vol': 0.028185785566678202,
 'median_profit_over_vol': 0.06400481930986958,
 'max_drawdown': -122815.61336517334}

In [8]:
bt.get_equity_statistics_return()

{'total_return': 1.4190785592269899,
 'mean_return': 0.0008837170518698433,
 'median_return': 0.0016624649558423243,
 'std_return': 0.01902566724206684,
 'mad_return': 0.014076790258608168,
 'skewness': -0.1545743314802025,
 'kurtosis': 4.070887089606387,
 'mean_median_gap': -0.0007787479039724811,
 'mean_return_over_vol': 0.0464486759190182,
 'median_return_over_vol': 0.08738011312247275,
 'max_drawdown': -0.34320915470344215}

In [9]:
bt.tradebook

,ID,STATUS,NOTES,PNL,CREATED_ITERATION,CLOSED_ITERATION,REASON_CLOSED,POSITIONS_TOTAL,STOP_LOSS,TAKE_PROFIT,MAX_AGE
0,86d99005-eb4b-4777-bd33-0d96266386e7,OPEN,Buy and Hold Trade,141907.855923,2018-01-02,None,None,241870.675186,None,None,None


In [10]:
bt.orderbook

,ID,TICKER,ACTION,STATUS,TRADE_ID,REQUESTED_ITERATION,REQUESTED_QUANTITY,REQUESTED_PRICE,REQUESTED_FEE,REQUESTED_GROSS_TOTAL,REQUESTED_TOTAL,FILLED_ITERATION,FILLED_QUANTITY,FILLED_PRICE,FILLED_FEE,FILLED_GROSS_TOTAL,FILLED_TOTAL
0,8c126eeb-0571-4053-aac2-7f4db271f194,AAPL,BUY,FILLED,86d99005-eb4b-4777-bd33-0d96266386e7,2018-01-02,1239,40.341881,0.0,49983.590309,49983.590309,2018-01-02,1239,40.341881,0.0,-49983.590309,-49983.590309
1,b4b64931-9e8c-435d-8f6f-ab5080e2c064,GOOG,BUY,FILLED,86d99005-eb4b-4777-bd33-0d96266386e7,2018-01-02,945,52.888073,0.0,49979.228954,49979.228954,2018-01-02,945,52.888073,0.0,-49979.228954,-49979.228954


In [11]:
bt.strategy.positions

{'AAPL': {'quantity': 1239, 'value': np.float64(158590.73332214355)},
 'GOOG': {'quantity': 945, 'value': np.float64(83279.94186401367)}}